# BNPL ML Model Development

**Objective**: Develop production-ready ML model for BNPL default risk prediction

**Performance Targets**:
- Beat current 3.5x risk discrimination baseline
- Achieve >40% precision on high-risk segment
- Maintain <100ms inference latency
- Production deployment ready

**Focus**: Primary target is `will_default` prediction (binary classification).

## Implementation Steps

### **Step 1: Algorithm Research & Selection** 
- Evaluate ML algorithms for BNPL risk assessment
- Consider inference latency requirements (<100ms)
- Assess interpretability for regulatory compliance
- Select candidate algorithms for testing

### **Step 2: Data Loading & Feature Engineering**
- Load engineered features using `BNPLFeatureEngineer` class
- Validate feature quality and distribution
- Prepare data for modeling

### **Step 3: Baseline Performance Establishment**
- Implement current underwriting baseline (3.5x discrimination)
- Train candidate models on engineered features
- Establish performance benchmarks

### **Step 4: Model Training & Evaluation**
- Cross-validation framework
- Business metrics: discrimination ratio, precision/recall
- Technical metrics: latency, model size, memory usage
- Feature importance analysis

### **Step 5: Production Readiness Assessment**
- Inference latency benchmarking
- Model serialization and deployment format
- Edge case handling and fallback strategies
- A/B testing framework preparation

### **Step 6: Model Selection & Recommendations**
- Compare algorithms across business and technical dimensions
- Select final model for production deployment
- Document trade-offs and deployment considerations
- Prepare ML engineering handoff

In [ ]:
# Add project root to Python path for imports
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    
print(f"Project root added to path: {project_root}")

In [ ]:
# Environment setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import (ExtraTreesClassifier, HistGradientBoostingClassifier,
                            VotingClassifier, StackingClassifier, AdaBoostClassifier,
                            GradientBoostingClassifier, BaggingClassifier)
from sklearn.naive_bayes import ComplementNB, BernoulliNB
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier


# Import models for training
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# For feature importance
from sklearn.inspection import permutation_importance


import joblib
import time
import warnings
warnings.filterwarnings('ignore')

# Import our custom feature engineering class
from flit_ml.features.bnpl_feature_engineering import BNPLFeatureEngineer

# Configuration
pd.set_option('display.max_columns', 50)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print("ML development environment ready!")
print(f"Available ML libraries loaded successfully")

## Step 1: Algorithm Research & Selection

### ML Algorithm Comparison for BNPL Risk Assessment

| Algorithm | Inference Speed | Interpretability | Performance | Memory Usage | Production Ready | Pros | Cons |
|-----------|----------------|------------------|-------------|--------------|------------------|------|------|
| **LogisticRegression** | Fastest (~1ms) | High (coefficients) | Good baseline | Minimal | Excellent | Fast inference, Interpretable, Stable, Low memory | Linear assumptions, May miss complex patterns |
| **RandomForestClassifier** | Fast (~5-10ms) | Medium (feature importance) | Strong for tabular | Moderate | Good | Handles non-linearity, Feature importance, Robust | Larger model size, Less interpretable |
| **XGBoost** | Fast (~10-20ms) | Medium (SHAP values) | Often best for tabular | Moderate | Good | High performance, Feature importance, Handles missing values | Hyperparameter tuning, Model complexity |
| **LightGBM** | Very Fast (~5ms) | Medium (SHAP values) | Excellent for tabular | Low | Excellent | Fast training/inference, Memory efficient, High performance | Can overfit small datasets, Less interpretable than linear |

### Selection Criteria for BNPL Production System

1. **Inference latency**: <100ms (preferably <20ms)
2. **Performance**: Beat 3.5x discrimination ratio
3. **Interpretability**: Regulatory compliance requirements
4. **Production stability**: Minimal maintenance overhead
5. **Memory efficiency**: Scalable serving architecture

### Recommended Starting Models

Based on BNPL constraints, we'll focus on all 4 models before considering more complex models.

## Step 2: Data Loading & Feature Engineering

Load engineered features using our production-ready `BNPLFeatureEngineer` class.

In [ ]:
# Initialize feature engineer with clean output for development                                                                                                                                       
feature_engineer = BNPLFeatureEngineer(verbose=True)                                                                                                                                               
                                                                                                                                                                                                        
# Load and engineer features                                                                                                                                                                          
print("🚀 Loading and engineering features for model development...")                                                                                                                                 
df_features, feature_metadata = feature_engineer.engineer_features(                                                                                                                                   
    sample_size=1000,  # Sample size for development                                                                                                                                                  
    random_seed=42                                                                                                                                                                                    
)                                                                                                                                                                                                     
                                                                                                                                                                                                      
print(f"\n📊 Feature Engineering Complete:")                                                                                                                                                          
print(f"   Dataset shape: {df_features.shape}")                                                                                                                                                       
print(f"   Features available: {len(feature_metadata['all_features'])}")                                                                                                                              
print(f"   Target variable: {feature_metadata['target_variable']}")                                                                                                                                   
print(f"   Default rate: {df_features['will_default'].mean():.1%}")

In [ ]:
# Validate feature quality and distribution
print("🔍 Feature Quality Assessment:")
print(f"   Data shape: {df_features.shape}")
print(f"   Memory usage: {df_features.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"   Default rate: {df_features['will_default'].mean():.1%}")

# Check for any remaining issues
missing_values = df_features.isnull().sum().sum()
print(f"   Missing values: {missing_values}")

# Feature type summary
print(f"\n📋 Feature Types:")
for feature_type, features in feature_metadata.items():
    if isinstance(features, list) and feature_type.endswith('_features'):
        print(f"   {feature_type}: {len(features)} features")

# Display sample of features
print(f"\n📋 Sample of engineered features:")
display_cols = df_features.columns[:10].tolist()
if 'will_default' not in display_cols:
    display_cols.append('will_default')
    
df_features[display_cols].head()

In [ ]:
# Prepare data for modeling
print("🔧 Preparing data for model training...")

# Separate features and target - exclude both target variables
exclude_cols = ['will_default', 'days_to_first_missed_payment']  # Remove both target variables, even though the 2nd one is secondary (for more enhanced models)
feature_cols = [col for col in df_features.columns if col not in exclude_cols]
numeric_features = ['amount', 'risk_score', 'payment_credit_limit', 'price_comparison_time', 'customer_tenure_days']
categorical_features = [col for col in feature_cols if col not in numeric_features]
primary_target_col = exclude_cols[0]  # 'will_default'
secondary_target_col = exclude_cols[1]  # 'days_to_first_missed_payment'

# For now, we will focus on the primary target
target_col = primary_target_col

X = df_features[feature_cols]
y = df_features[target_col]

print(f"   Features shape: {X.shape}")
print(f"   Target shape: {y.shape}")
print(f"   Target distribution: {y.value_counts().to_dict()}")
print(f"   Class balance: {y.mean():.1%} positive class")

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"\n📊 Train-Test Split:")
print(f"   Training set: {X_train.shape}")
print(f"   Test set: {X_test.shape}")
print(f"   Train default rate: {y_train.mean():.1%}")
print(f"   Test default rate: {y_test.mean():.1%}")

In [ ]:
# Feature scaling for algorithms that need it
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)


 # Create preprocessor
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),      # Scale numeric features
    ('cat', 'passthrough', categorical_features)      # Leave categorical unchanged
])

X_train_processed = preprocessor.fit_transform(X_train)  # Fit AND transform train
X_test_processed = preprocessor.transform(X_test)        # Only transform test (using train's fit) --> Avoid inconsistent scaling train vs test


print(f"\n✅ Data preparation complete")
print(f"   Ready for model training and evaluation")

## Step 3: Baseline Performance Establishment

Implement current underwriting baseline and train candidate models.

In [ ]:
# Current underwriting baseline analysis
print("📊 Current Underwriting Baseline Analysis")
print("=" * 45)

# Analyze current risk_score and risk_level performance
if 'risk_score' in df_features.columns and 'risk_level_encoded' in df_features.columns:
    
    # Risk level performance
    risk_performance = df_features.groupby('risk_level_encoded')['will_default'].agg(['count', 'mean']).round(3)
    risk_performance.columns = ['transaction_count', 'default_rate']
    
    print(f"\n🎯 Current Risk Level Performance:")
    risk_level_mapping = {0: 'Low', 1: 'Medium', 2: 'High'}
    for idx, row in risk_performance.iterrows():
        level_name = risk_level_mapping.get(idx, f'Level_{idx}')
        print(f"   {level_name} Risk: {row['default_rate']:.1%} default rate ({row['transaction_count']:,} transactions)")
    
    # Calculate discrimination ratio
    high_risk_rate = risk_performance.loc[2, 'default_rate']  # High risk (encoded as 2)
    low_risk_rate = risk_performance.loc[0, 'default_rate']   # Low risk (encoded as 0)
    current_discrimination = high_risk_rate / low_risk_rate if low_risk_rate > 0 else 0
    
    print(f"\n📈 Current System Discrimination:")
    print(f"   High risk default rate: {high_risk_rate:.1%}")
    print(f"   Low risk default rate: {low_risk_rate:.1%}")
    print(f"   Discrimination ratio: {current_discrimination:.1f}x")
    print(f"   Target to beat: >{current_discrimination:.1f}x")
    
    # Risk score distribution analysis
    print(f"\n📊 Risk Score Distribution:")
    risk_score_stats = df_features['risk_score'].describe()
    print(f"   Range: {risk_score_stats['min']:.2f} - {risk_score_stats['max']:.2f}")
    print(f"   Mean: {risk_score_stats['mean']:.2f}")
    print(f"   Std: {risk_score_stats['std']:.2f}")
    
else:
    print("⚠️  Risk score/level features not available in dataset")
    current_discrimination = 3.5  # Use known baseline from context
    print(f"   Using known baseline discrimination ratio: {current_discrimination}x")

# Set performance targets
target_discrimination = max(current_discrimination * 1.1, 4.0)  # At least 10% improvement or 4.0x
target_precision = 0.40  # 40% precision on high-risk segment

print(f"\n🎯 Performance Targets for ML Models:")
print(f"   Discrimination ratio: >{target_discrimination:.1f}x")
print(f"   High-risk precision: >{target_precision:.0%}")
print(f"   Inference latency: <100ms")
print(f"   Model size: <50MB (for fast loading)")

## Step 4: Model Training & Cross-Validation

Train candidate models with proper preprocessing and evaluate using cross-validation.

In [ ]:
# Initialize models for comparison
models = {
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
    'RandomForest': RandomForestClassifier(random_state=42, n_estimators=100),
    'LightGBM': LGBMClassifier(random_state=42, verbosity=-1),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss')
}

print(f"🤖 Models initialized: {list(models.keys())}")

# Cross-validation setup
cv_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
print(f"📊 Cross-validation: {cv_folds.n_splits}-fold stratified")

In [ ]:
cv_results = {}

print("🚀 Starting model training and cross-validation...")
print("=" * 60)

for model_name, model in models.items():
    print(f"\n🔧 Training {model_name}...")

    # Use appropriate data (scaled for LR, unscaled for tree-based)
    if model_name == 'LogisticRegression':
        X_train_model = X_train_processed
        X_test_model = X_test_processed
    else:
        # Tree-based models don't need scaling
        X_train_model = X_train.values
        X_test_model = X_test.values

    # Cross-validation
    start_time = time.time()
    cv_scores = cross_val_score(model, X_train_model, y_train, cv=cv_folds, scoring='roc_auc')
    cv_time = time.time() - start_time

    # Train on full training set
    start_time = time.time()
    model.fit(X_train_model, y_train)
    train_time = time.time() - start_time

    # Store results
    cv_results[model_name] = {
        'cv_scores': cv_scores,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'cv_time': cv_time,
        'train_time': train_time,
        'model': model
    }

    print(f"   CV ROC-AUC: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
    print(f"   CV Time: {cv_time:.2f}s, Train Time: {train_time:.2f}s")

print(f"\n✅ Model training complete!")

Initial model training gives AUC of ~0.5, showing poor performance across models. Let's investigate this:

In [ ]:
X_train_processed

In [ ]:
# Check correlations with target variable
print("🔍 Feature-Target Correlations:")
target_corrs = df_features.corr()['will_default'].drop(['will_default','days_to_first_missed_payment']).sort_values(key=abs, ascending=False)
print(target_corrs.head(10))

# Check if we have any strong correlations
strong_corrs = target_corrs[abs(target_corrs) > 0.1]
print(f"\nFeatures with |correlation| > 0.1: {len(strong_corrs)}")
if len(strong_corrs) > 0:
    print(strong_corrs)
else:
    print("⚠️ No features have strong correlation with target!")

# Correlation heatmap
plt.figure(figsize=(12, 10))
# Focus on features most correlated with target
top_features = target_corrs.head(15).index.tolist() + ['will_default']
corr_matrix = df_features[top_features].corr()

sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0,
            fmt='.2f', square=True, cbar_kws={'label': 'Correlation'})
plt.title('Feature Correlation Heatmap (Top 15 vs Target)')
plt.tight_layout()
plt.show()

# Quick sanity checks
print(f"\n📊 Data Sanity Checks:")
print(f"Target distribution: {y.value_counts().to_dict()}")
print(f"Features shape: {X.shape}")
print(f"Any missing values in X: {X.isnull().sum().sum()}")
print(f"Feature columns: {list(X.columns[:5])}...")

In [ ]:
print("X_train columns:", list(X_train.columns))
print("X_train shape:", X_train.shape)


# Correlations for features actually used in training
train_data_with_target = pd.concat([X_train, y_train], axis=1)
actual_corrs = train_data_with_target.corr()['will_default'].drop('will_default').sort_values(key=abs, ascending=False)
print("Actual training feature correlations:")
print(actual_corrs.head(10))


print("Are we using X_train for tree models?", type(X_train))
print("Are we using X_train_processed for LogReg?", type(X_train_processed))

In [ ]:
train_data_with_target = pd.concat([X_train, y_train], axis=1)
actual_corrs = train_data_with_target.corr()['will_default'].drop('will_default').sort_values(key=abs, ascending=False)

print("🔍 ACTUAL Training Feature Correlations:")
print(actual_corrs.head(10))

print(f"\nFeatures with |correlation| > 0.05:")
strong_corrs = actual_corrs[abs(actual_corrs) > 0.05]
print(strong_corrs)

print(f"\nX_train columns: {list(X_train.columns)}")
print(f"X_train shape: {X_train.shape}")

In [ ]:
# Test if current risk_level shows expected discrimination
print("Risk level performance check:")
risk_perf = df_features.groupby('risk_level_encoded')['will_default'].mean()
print(risk_perf)

In [ ]:
print("Features removed during cleaning:")
removed_features = feature_metadata.get('features_removed', [])
for feature in removed_features:
    print(f"  - {feature}")

print(f"\nTotal removed: {len(removed_features)}")

The first 4 models gave a performance as poor as random, and yet there seems to be no obvious reason apart from the poor correlation btn features and the predicted. So this is mostly a data quality issue --  our data is not predictive or it's just plain unrealistic (again, feedback for Simtom). Question is, though, it is possible to still get some predictive power from features that have such low correlation levels to our target variable? To determine this, we will brute-force our eval of models and just throw many models at the problem to see which one likely gives something reasonable.

In [ ]:
# Complete model portfolio - All Tiers
models = {
    # Original models
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
    'RandomForest': RandomForestClassifier(random_state=42, n_estimators=100),
    'LightGBM': LGBMClassifier(random_state=42, verbosity=-1),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss'),

    # Tier 1: Must Try
    'HistGradientBoosting': HistGradientBoostingClassifier(random_state=42),
    'ExtraTrees': ExtraTreesClassifier(random_state=42, n_estimators=100),
    #'ComplementNB': ComplementNB(),

    # Tier 2: High Value
    'Ridge': RidgeClassifier(random_state=42),
    'ElasticNet': LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga',
                                    random_state=42, max_iter=1000),
    #'SVM_Linear': SVC(kernel='linear', probability=True, random_state=42),
    #'MLPClassifier': MLPClassifier(hidden_layer_sizes=(100, 50), random_state=42, max_iter=500),

    # Tier 3: Experimental  
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'KNN_Weighted': KNeighborsClassifier(n_neighbors=10, weights='distance'),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'BernoulliNB': BernoulliNB(),
    'SGDClassifier': SGDClassifier(random_state=42, loss='log_loss', max_iter=1000),
    'BaggingClassifier': BaggingClassifier(random_state=42),
    'SVM_RBF': SVC(kernel='rbf', probability=True, random_state=42),
}

print(f"🤖 Complete model portfolio: {len(models)} algorithms")
print("Tier breakdown:")
print("  Original: LogisticRegression, RandomForest, LightGBM, XGBoost")
print("  Tier 1: HistGradientBoosting, ExtraTrees, ComplementNB")
print("  Tier 2: Ridge, ElasticNet, SVM_Linear, MLPClassifier")
print("  Tier 3: KNN variants, AdaBoost, GradientBoosting, BernoulliNB, SGD, Bagging, SVM_RBF")

In [ ]:
# Enhanced training with proper error handling and model-specific logic
cv_results = {}

# Model-specific configurations
model_configs = {
    'SVM_RBF': {'max_samples': 10000},  # Subsample for speed
    'SVM_Linear': {'max_samples': 20000},
    'KNN': {'max_samples': 15000},
    'KNN_Weighted': {'max_samples': 15000},
}

print("🚀 Starting robust model training and evaluation...")
print(f"Training {len(models)} models on {X_train.shape[0]:,} samples")
print("=" * 70)

for model_name, model in models.items():
    print(f"\n🔧 Training {model_name}...")

    # Determine preprocessing needs
    scaling_models = ['LogisticRegression', 'ElasticNet', 'SVM_Linear', 'SVM_RBF',
                    'MLPClassifier', 'KNN', 'KNN_Weighted', 'SGDClassifier','Ridge']

    # Prepare data
    if model_name in scaling_models:
        X_train_model = X_train_processed
        X_test_model = X_test_processed
        data_type = "scaled"
    else:
        X_train_model = X_train
        X_test_model = X_test
        data_type = "unscaled"

    # Apply model-specific sampling if needed
    if model_name in model_configs:
        max_samples = model_configs[model_name].get('max_samples')
        if max_samples and len(X_train_model) > max_samples:
            from sklearn.model_selection import train_test_split
            X_sample, _, y_sample, _ = train_test_split(
                X_train_model, y_train, train_size=max_samples,
                stratify=y_train, random_state=42
            )
            X_train_model, y_train_model = X_sample, y_sample
            print(f"   Using subsample: {len(X_train_model):,} samples")
        else:
            y_train_model = y_train
    else:
        y_train_model = y_train

    try:
        # Cross-validation with timeout
        start_time = time.time()
        cv_scores = cross_val_score(model, X_train_model, y_train_model,
                                    cv=cv_folds, scoring='roc_auc', n_jobs=1)
        cv_time = time.time() - start_time

        # Skip if CV takes too long
        if cv_time > 300:  # 5 minutes timeout
            print(f"   ⏰ Timeout: CV took {cv_time:.1f}s, skipping")
            continue

        # Train on full training set
        start_time = time.time()
        model.fit(X_train_model, y_train_model)
        train_time = time.time() - start_time

        # Test predictions with proper handling
        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_test_model)[:, 1]
        elif hasattr(model, 'decision_function'):
            # For Ridge, SVM without probability
            y_pred_proba = model.decision_function(X_test_model)
        else:
            print(f"   ⚠️ No probability/decision function available")
            continue

        test_auc = roc_auc_score(y_test, y_pred_proba)

        # Store results
        cv_results[model_name] = {
            'cv_scores': cv_scores,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'test_auc': test_auc,
            'cv_time': cv_time,
            'train_time': train_time,
            'model': model,
            'data_type': data_type,
            'status': 'success',
            'samples_used': len(X_train_model)
        }

        print(f"   CV ROC-AUC: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
        print(f"   Test ROC-AUC: {test_auc:.3f}")
        print(f"   Times: CV={cv_time:.1f}s, Train={train_time:.2f}s")

    except Exception as e:
        print(f"   ❌ Failed: {str(e)[:100]}")
        cv_results[model_name] = {'status': 'failed', 'error': str(e)}

print(f"\n✅ Robust training complete!")

In [ ]:
# Extract successful results
successful_models = {k: v for k, v in cv_results.items() if v.get('status') == 'success'}

if len(successful_models) == 0:
    print("❌ No successful models to plot!")
else:
    # Create subplots for multiple comparisons
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Model Performance Comparison Dashboard', fontsize=16, fontweight='bold')

    # 1. ROC-AUC Performance Comparison
    ax1 = axes[0, 0]
    model_names = list(successful_models.keys())
    test_aucs = [successful_models[name]['test_auc'] for name in model_names]
    cv_means = [successful_models[name]['cv_mean'] for name in model_names]
    cv_stds = [successful_models[name]['cv_std'] for name in model_names]

    # Sort by test performance
    sorted_indices = sorted(range(len(test_aucs)), key=lambda i: test_aucs[i], reverse=True)
    model_names_sorted = [model_names[i] for i in sorted_indices]
    test_aucs_sorted = [test_aucs[i] for i in sorted_indices]
    cv_means_sorted = [cv_means[i] for i in sorted_indices]
    cv_stds_sorted = [cv_stds[i] for i in sorted_indices]

    x_pos = range(len(model_names_sorted))
    bars1 = ax1.bar(x_pos, test_aucs_sorted, alpha=0.7, label='Test ROC-AUC', color='steelblue')
    bars2 = ax1.bar([x + 0.4 for x in x_pos], cv_means_sorted, alpha=0.7,
                    yerr=cv_stds_sorted, label='CV ROC-AUC (±std)', color='orange', width=0.4)

    ax1.axhline(y=0.5, color='red', linestyle='--', alpha=0.7, label='Random (0.5)')
    ax1.set_xlabel('Models (sorted by Test performance)')
    ax1.set_ylabel('ROC-AUC Score')
    ax1.set_title('Model ROC-AUC Performance Comparison')
    ax1.set_xticks([x + 0.2 for x in x_pos])
    ax1.set_xticklabels(model_names_sorted, rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Add value labels on bars
    for i, (bar1, bar2) in enumerate(zip(bars1, bars2)):
        ax1.text(bar1.get_x() + bar1.get_width()/2, bar1.get_height() + 0.01,
                f'{test_aucs_sorted[i]:.3f}', ha='center', va='bottom', fontsize=8)

    # 2. Training Time vs Performance
    ax2 = axes[0, 1]
    train_times = [successful_models[name]['train_time'] for name in model_names]
    colors = ['red' if auc < 0.55 else 'orange' if auc < 0.65 else 'green'
            for auc in test_aucs]

    scatter = ax2.scatter(train_times, test_aucs, c=colors, s=100, alpha=0.7)
    for i, name in enumerate(model_names):
        ax2.annotate(name, (train_times[i], test_aucs[i]),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)

    ax2.axhline(y=0.5, color='red', linestyle='--', alpha=0.7)
    ax2.set_xlabel('Training Time (seconds)')
    ax2.set_ylabel('Test ROC-AUC')
    ax2.set_title('Performance vs Training Speed')
    ax2.grid(True, alpha=0.3)

    # 3. Cross-Validation Consistency (CV mean vs std)
    ax3 = axes[1, 0]
    cv_consistency = ax3.scatter(cv_means, cv_stds, s=100, alpha=0.7, c=test_aucs,
                                cmap='RdYlGn', vmin=0.4, vmax=0.8)
    for i, name in enumerate(model_names):
        ax3.annotate(name, (cv_means[i], cv_stds[i]),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)

    ax3.axvline(x=0.5, color='red', linestyle='--', alpha=0.7)
    ax3.set_xlabel('CV Mean ROC-AUC')
    ax3.set_ylabel('CV Standard Deviation')
    ax3.set_title('Model Consistency (Lower std = more stable)')
    ax3.grid(True, alpha=0.3)
    plt.colorbar(cv_consistency, ax=ax3, label='Test ROC-AUC')

    # 4. Top Models Detailed Comparison
    ax4 = axes[1, 1]
    # Show top 6 models
    top_n = min(6, len(model_names_sorted))
    top_models = model_names_sorted[:top_n]
    top_test_aucs = test_aucs_sorted[:top_n]
    top_cv_means = cv_means_sorted[:top_n]
    top_cv_stds = cv_stds_sorted[:top_n]

    x_pos_top = range(top_n)
    ax4.barh(x_pos_top, top_test_aucs, alpha=0.7, color='steelblue', label='Test ROC-AUC')
    ax4.errorbar(top_cv_means, x_pos_top, xerr=top_cv_stds, fmt='o',
                color='orange', label='CV ROC-AUC (±std)')

    ax4.axvline(x=0.5, color='red', linestyle='--', alpha=0.7, label='Random')
    ax4.set_yticks(x_pos_top)
    ax4.set_yticklabels(top_models)
    ax4.set_xlabel('ROC-AUC Score')
    ax4.set_title(f'Top {top_n} Models - Detailed View')
    ax4.legend()
    ax4.grid(True, alpha=0.3)

    # Add value labels
    for i, auc in enumerate(top_test_aucs):
        ax4.text(auc + 0.01, i, f'{auc:.3f}', va='center', fontsize=9)

    plt.tight_layout()
    plt.show()

# Summary statistics table
print(f"\n📊 Performance Summary Table")
print("="*80)
print(f"{'Model':<20} {'Test AUC':<10} {'CV Mean':<10} {'CV Std':<10} {'Train Time':<12}")
print("-"*80)
for name in model_names_sorted:
    results = successful_models[name]
    print(f"{name:<20} {results['test_auc']:<10.3f} {results['cv_mean']:<10.3f} "
        f"{results['cv_std']:<10.3f} {results['train_time']:<12.2f}s")

# Quick insights
best_model = model_names_sorted[0]
print(f"\n🏆 Best performing model: {best_model} (AUC: {successful_models[best_model]['test_auc']:.3f})")

breakthrough_models = [name for name, results in successful_models.items()
                    if results['test_auc'] > 0.55]
if breakthrough_models:
    print(f"🚀 Models breaking through 0.55 barrier: {len(breakthrough_models)}")
    for name in breakthrough_models:
        print(f"   - {name}: {successful_models[name]['test_auc']:.3f}")
else:
    print("⚠️  No models achieved AUC > 0.55")

In [ ]:
# 1. Select Top Models for Tuning
# Extract top 5 models by performance with speed considerations
top_models_for_tuning = []
for name in model_names_sorted[:8]:  # Consider top 8
    results = successful_models[name]
    # Balance: AUC > 0.6 AND training time < 5s (adjustable threshold)
    if results['test_auc'] >= 0.59 and results['train_time'] < 5.0:
        top_models_for_tuning.append((name, results))

print("🎯 Selected models for hyperparameter tuning:")
for name, results in top_models_for_tuning[:10]:
    print(f"  {name}: AUC={results['test_auc']:.3f}, Time={results['train_time']:.2f}s")

In [ ]:
# Hyperparameter grids focusing on speed AND performance
param_grids = {
    'GradientBoosting': {
        'n_estimators': [50, 100],  # Reduce from default for speed
        'learning_rate': [0.1, 0.15, 0.2],  # Higher LR = fewer iterations
        'max_depth': [3, 5],  # Shallower trees = faster
        'subsample': [0.8, 1.0],  # Subsampling for speed
    },

    'Ridge': {
        'alpha': [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0],
        'solver': ['auto', 'svd', 'cholesky', 'lsqr'],
        'fit_intercept': [True, False]
    },

    'ElasticNet': {
        'C': [0.1, 1.0, 10.0, 100.0],
        'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
        'max_iter': [500, 1000],  # Reduce iterations for speed
        'tol': [1e-3, 1e-4]  # Relaxed tolerance for speed
    },

    'LogisticRegression': {
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['liblinear', 'saga'],
        'max_iter': [1000, 2000],
        'tol': [1e-3, 1e-4]
    },

    'BernoulliNB': {
        'alpha': [0.01, 0.1, 0.5, 1.0, 2.0],
        'fit_prior': [True, False],
        'binarize': [0.0, 0.1, 0.5]
    },

    'AdaBoost': {
        'n_estimators': [50, 100, 200],  # Test if more helps
        'learning_rate': [0.5, 1.0, 1.5],
        'algorithm': ['SAMME', 'SAMME.R']
    },

    'HistGradientBoosting': {
        'learning_rate': [0.05, 0.1, 0.15, 0.2],
        'max_iter': [100, 200, 300],
        'max_depth': [3, 6, 10, None],
        'min_samples_leaf': [5, 10, 20]
    },

    'SGDClassifier': {
        'alpha': [0.0001, 0.001, 0.01, 0.1],
        'learning_rate': ['constant', 'optimal', 'invscaling'],
        'eta0': [0.01, 0.1, 1.0],
        'max_iter': [500, 1000],
        'tol': [1e-3, 1e-4]
    },

    'LightGBM': {
        'n_estimators': [100, 200, 300],
        'num_leaves': [20, 31, 50],
        'learning_rate': [0.05, 0.1, 0.15],
        'min_child_samples': [10, 20],
        'subsample': [0.8, 1.0]
    }
}

print("🎯 Tuning all 9 models with AUC ≥ 0.6")
print("Focus areas:")
print("  Speed optimization: GradientBoosting, ElasticNet")
print("  Performance boost: Ridge, LogisticRegression, BernoulliNB, HistGB, SGD")
print("  Push over threshold: LightGBM (0.599 → 0.6+)")
print("  Balance optimization: AdaBoost")

In [ ]:
# 3. Efficient Tuning Strategy

tuned_models = {}

print("🔧 Starting hyperparameter tuning for top models...")
print("="*60)

for model_name, _ in top_models_for_tuning:
    if model_name not in param_grids:
        continue

    print(f"\n🎯 Tuning {model_name}...")

    # Get base model and data
    base_model = models[model_name]
    if model_name in scaling_models:
        X_tune, y_tune = X_train_processed, y_train
    else:
        X_tune, y_tune = X_train.values, y_train

    try:
        # Use RandomizedSearchCV for speed
        search = RandomizedSearchCV(
            base_model,
            param_grids[model_name],
            n_iter=25,  # Increased for better coverage
            cv=3,       # 3-fold for speed
            scoring='roc_auc',
            n_jobs=-1,
            random_state=42,
            verbose=0
        )

        start_time = time.time()
        search.fit(X_tune, y_tune)
        tune_time = time.time() - start_time

        # Test best model with proper probability handling
        best_model = search.best_estimator_
        if model_name in scaling_models:
            X_test_model = X_test_processed
        else:
            X_test_model = X_test.values

        # Handle different model prediction methods
        if hasattr(best_model, 'predict_proba'):
            y_pred_proba = best_model.predict_proba(X_test_model)[:, 1]
        elif hasattr(best_model, 'decision_function'):
            # For Ridge, SVM without probability
            y_pred_proba = best_model.decision_function(X_test_model)
        else:
            print(f"   ⚠️  No probability method available for {model_name}")
            continue

        tuned_auc = roc_auc_score(y_test, y_pred_proba)
        baseline_auc = successful_models[model_name]['test_auc']
        improvement = tuned_auc - baseline_auc

        tuned_models[model_name] = {
            'model': best_model,
            'best_params': search.best_params_,
            'best_cv_score': search.best_score_,
            'test_auc': tuned_auc,
            'baseline_auc': baseline_auc,
            'improvement': improvement,
            'tune_time': tune_time,
            'status': 'success'
        }

        print(f"   Best params: {search.best_params_}")
        print(f"   Baseline AUC: {baseline_auc:.3f} → Tuned AUC: {tuned_auc:.3f}")
        print(f"   Improvement: {improvement:+.3f} ({tune_time:.1f}s tuning)")

    except Exception as e:
        print(f"   ❌ Tuning failed: {str(e)[:100]}")
        tuned_models[model_name] = {'status': 'failed', 'error': str(e)}

In [ ]:
# Summary of tuning results
successful_tuning = {k: v for k, v in tuned_models.items() if v.get('status') == 'success'}
failed_tuning = {k: v for k, v in tuned_models.items() if v.get('status') == 'failed'}

print(f"\n📈 Hyperparameter Tuning Results:")
print("="*60)
if successful_tuning:
    for name, results in sorted(successful_tuning.items(), key=lambda x: x[1]['test_auc'], reverse=True):
        print(f"{name:20} {results['baseline_auc']:.3f} → {results['test_auc']:.3f} "
            f"({results['improvement']:+.3f}) [{results['tune_time']:.1f}s]")

    # Best tuned model
    best_tuned = max(successful_tuning.items(), key=lambda x: x[1]['test_auc'])
    print(f"\n🏆 Best tuned model: {best_tuned[0]} (AUC: {best_tuned[1]['test_auc']:.3f})")

    # Biggest improvement
    best_improvement = max(successful_tuning.items(), key=lambda x: x[1]['improvement'])
    print(f"📈 Biggest improvement: {best_improvement[0]} ({best_improvement[1]['improvement']:+.3f})")

if failed_tuning:
    print(f"\n❌ Failed tuning: {list(failed_tuning.keys())}")

# Compare with original baseline
print(f"\n🆚 Best Models Comparison:")
print("="*50)
print("ORIGINAL BEST:")
orig_best = model_names_sorted[0]
print(f"  {orig_best}: {successful_models[orig_best]['test_auc']:.3f}")

if successful_tuning:
    tuned_best = best_tuned[0]
    print("AFTER TUNING:")
    print(f"  {tuned_best}: {best_tuned[1]['test_auc']:.3f}")
    overall_improvement = best_tuned[1]['test_auc'] - successful_models[orig_best]['test_auc']
    print(f"OVERALL IMPROVEMENT: {overall_improvement:+.3f}")

### Evaluating Model against Business Usefulness

Having evaluated the top models and determined that Ridge is best, we will proceed to do a business assessment to see if this model has any business value. If there is value, however little, we will ship to prod and then later improve the quality of the model and the quality of the data.

In [ ]:
def evaluate_business_metrics(model, X_test_data, y_test, model_name="Best Model"):
    """
    Evaluate model using BNPL-specific business metrics
    
    Args:
        model: Trained model to evaluate
        X_test_data: Test features (ensure correct scaling for model type)
        y_test: Test targets
        model_name: Name for display
    """

    print(f"📊 Business Metrics Evaluation: {model_name}")
    print("="*60)

    # Generate predictions
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test_data)[:, 1]
        pred_type = "predict_proba"
    elif hasattr(model, 'decision_function'):
        y_pred_proba = model.decision_function(X_test_data)
        pred_type = "decision_function"
    else:
        raise ValueError("Model has no probability/decision function")

    print(f"Prediction method: {pred_type}")
    print(f"Data shape: {X_test_data.shape}")

    # 1. Overall ROC-AUC
    auc_score = roc_auc_score(y_test, y_pred_proba)
    print(f"ROC-AUC Score: {auc_score:.3f}")

    # 2. Risk-based segmentation (equal-sized tertiles)
    try:
        risk_segments_equal = pd.qcut(y_pred_proba, q=3, labels=['Low', 'Medium', 'High'])
        segment_perf_equal = pd.DataFrame({
            'segment': risk_segments_equal,
            'actual_default': y_test.values
        }).groupby('segment')['actual_default'].agg(['count', 'mean']).round(3)

        discrimination_equal = segment_perf_equal.loc['High', 'mean'] / segment_perf_equal.loc['Low', 'mean']

        print(f"\n1️⃣ Equal-Sized Risk Segments:")
        print(segment_perf_equal)
        print(f"   Discrimination Ratio: {discrimination_equal:.1f}x")

    except Exception as e:
        print(f"\n1️⃣ Equal-Sized Risk Segments: Error - {e}")
        discrimination_equal = 0.0

    # 3. Top 10% highest risk (more business-relevant)
    try:
        top_10_threshold = np.percentile(y_pred_proba, 90)
        top_10_mask = y_pred_proba >= top_10_threshold

        top_10_precision = y_test.values[top_10_mask].mean()
        bottom_90_rate = y_test.values[~top_10_mask].mean()
        discrimination_top10 = top_10_precision / bottom_90_rate if bottom_90_rate > 0 else float('inf')

        print(f"\n2️⃣ Top 10% Highest Risk Analysis:")
        print(f"   Top 10% default rate: {top_10_precision:.1%}")
        print(f"   Bottom 90% default rate: {bottom_90_rate:.1%}")
        print(f"   Discrimination Ratio: {discrimination_top10:.1f}x")
        print(f"   Top 10% precision: {top_10_precision:.1%}")

    except Exception as e:
        print(f"\n2️⃣ Top 10% Analysis: Error - {e}")
        discrimination_top10 = 0.0
        top_10_precision = 0.0

    # 4. Compare to baseline system performance
    print(f"\n3️⃣ vs Baseline System:")
    print(f"   Current system discrimination: 2.8x")
    print(f"   ML model discrimination (top 10%): {discrimination_top10:.1f}x")
    improvement = discrimination_top10 - 2.8
    print(f"   Improvement: {improvement:+.1f}x ({improvement/2.8*100:+.1f}%)")

    # 5. Business impact assessment
    if discrimination_top10 > 4.0:
        status = "🎯 EXCEEDS TARGET (>4.0x)"
    elif discrimination_top10 > 2.8:
        status = "✅ BEATS BASELINE (>2.8x)"
    else:
        status = "❌ BELOW BASELINE (<2.8x)"

    print(f"\n4️⃣ Business Assessment: {status}")

    # Return key metrics
    return {
        'auc': auc_score,
        'discrimination_equal': discrimination_equal,
        'discrimination_top10': discrimination_top10,
        'top10_precision': top_10_precision,
        'improvement_vs_baseline': improvement,
        'beats_baseline': discrimination_top10 > 2.8,
        'exceeds_target': discrimination_top10 > 4.0
    }

# Helper function to use correct data for each model type
def evaluate_model_with_correct_data(model_name, model_dict_key='tuned'):
    """Evaluate model with appropriate data scaling"""

    if model_dict_key == 'tuned' and model_name in tuned_models:
        model = tuned_models[model_name]['model']
        source = "tuned"
    elif model_name in successful_models:
        model = successful_models[model_name]['model']
        source = "original"
    else:
        print(f"❌ Model {model_name} not found")
        return None

    # Determine correct data type
    scaling_models = ['Ridge', 'LogisticRegression', 'ElasticNet', 'SVM_Linear', 'SVM_RBF',
                    'MLPClassifier', 'KNN', 'KNN_Weighted', 'SGDClassifier']

    if model_name in scaling_models:
        X_eval = X_test_processed
        data_info = "scaled"
    else:
        X_eval = X_test.values
        data_info = "unscaled"

    print(f"🔧 Evaluating {model_name} ({source} model) with {data_info} data")

    return evaluate_business_metrics(model, X_eval, y_test, f"{model_name} ({source})")

In [ ]:
# Ridge with correct scaling
ridge_metrics = evaluate_model_with_correct_data('Ridge', 'tuned')
ridge_metrics = evaluate_model_with_correct_data('Ridge', 'tuned')

In [ ]:
best_model = tuned_models['Ridge']['model']  # or whatever your best model is
business_metrics = evaluate_business_metrics(best_model, X_test.values, y_test, "Ridge Tuned")

## Advanced Ensemble Methods for BNPL Default Prediction

In [ ]:

from sklearn.ensemble import VotingClassifier, StackingClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
import time

print("🎯 Advanced Ensemble Methods Training (Fast Version)")
print("="*60)

In [ ]:
# Fast individual models for ensembles (replaced HistGB with SGD)
best_individual_models = [
    ('ridge', RidgeClassifier(alpha=10.0, solver='cholesky', random_state=42)),
    ('lgb', LGBMClassifier(random_state=42, verbosity=-1)),
    ('xgb', XGBClassifier(random_state=42, eval_metric='logloss')),
    ('sgd', SGDClassifier(random_state=42, loss='log_loss', max_iter=1000)),
    ('ada', AdaBoostClassifier(random_state=42))
]
# Create ensemble models
ensemble_models = {}

print("🔧 Creating fast ensemble models...")

In [ ]:
# 1. Voting Classifier (Soft Voting)
print("\n1️⃣ Training Voting Classifier (Soft)...")
# Only use models with predict_proba for soft voting
voting_models = [
    ('lgb', LGBMClassifier(random_state=42, verbosity=-1)),
    ('xgb', XGBClassifier(random_state=42, eval_metric='logloss')),
    ('sgd', SGDClassifier(random_state=42, loss='log_loss', max_iter=1000)),
    ('ada', AdaBoostClassifier(random_state=42))
]

voting_soft = VotingClassifier(
    estimators=voting_models,
    voting='soft',  # Average predicted probabilities
    n_jobs=-1
)

start_time = time.time()
voting_soft.fit(X_train.values, y_train)
voting_soft_time = time.time() - start_time

# Evaluate
y_pred_voting = voting_soft.predict_proba(X_test.values)[:, 1]
voting_soft_auc = roc_auc_score(y_test, y_pred_voting)

ensemble_models['VotingSoft'] = {
    'model': voting_soft,
    'auc': voting_soft_auc,
    'train_time': voting_soft_time,
    'data_type': 'unscaled'
}

print(f"   Voting Soft AUC: {voting_soft_auc:.3f} (Time: {voting_soft_time:.1f}s)")

In [ ]:
# 2. Voting Classifier (Hard Voting)
print("\n2️⃣ Training Voting Classifier (Hard)...")
voting_hard = VotingClassifier(
    estimators=best_individual_models,
    voting='hard',  # Majority vote
    n_jobs=-1
)

start_time = time.time()
voting_hard.fit(X_train.values, y_train)
voting_hard_time = time.time() - start_time

# For AUC calculation, get individual predictions and average
individual_preds = []
for name, model in voting_models:
    model.fit(X_train.values, y_train)
    if hasattr(model, 'predict_proba'):
        pred = model.predict_proba(X_test.values)[:, 1]
    elif hasattr(model, 'decision_function'):
        pred = model.decision_function(X_test.values)
    individual_preds.append(pred)

y_pred_voting_hard = np.mean(individual_preds, axis=0)
voting_hard_auc = roc_auc_score(y_test, y_pred_voting_hard)

ensemble_models['VotingHard'] = {
    'model': voting_hard,
    'auc': voting_hard_auc,
    'train_time': voting_hard_time,
    'data_type': 'unscaled'
}

print(f"   Voting Hard AUC: {voting_hard_auc:.3f} (Time: {voting_hard_time:.1f}s)")


In [ ]:
# 3. Stacking Classifier (Fast Version)
print("\n3️⃣ Training Stacking Classifier (Fast)...")
stacking_base_models = [
    ('lgb', LGBMClassifier(random_state=42, verbosity=-1, n_estimators=50)),  # Reduced trees
    ('xgb', XGBClassifier(random_state=42, eval_metric='logloss', n_estimators=50)),
    ('sgd', SGDClassifier(random_state=42, loss='log_loss', max_iter=500)),  # Fast SGD
    ('ada', AdaBoostClassifier(random_state=42, n_estimators=50))
]

stacking_clf = StackingClassifier(
    estimators=stacking_base_models,
    final_estimator=LogisticRegression(random_state=42, max_iter=500),  # Fast meta-learner
    cv=3,  # 3-fold CV for speed
    n_jobs=-1,
    passthrough=False
)

start_time = time.time()
stacking_clf.fit(X_train.values, y_train)
stacking_time = time.time() - start_time

y_pred_stacking = stacking_clf.predict_proba(X_test.values)[:, 1]
stacking_auc = roc_auc_score(y_test, y_pred_stacking)

ensemble_models['Stacking'] = {
    'model': stacking_clf,
    'auc': stacking_auc,
    'train_time': stacking_time,
    'data_type': 'unscaled'
}

print(f"   Stacking AUC: {stacking_auc:.3f} (Time: {stacking_time:.1f}s)")


In [ ]:
# 4. Bagging with LightGBM (Fast)
print("\n4️⃣ Training Bagging LightGBM (Fast)...")

bagging_lgb = BaggingClassifier(
    estimator=LGBMClassifier(random_state=42, verbosity=-1, n_estimators=30),  # Smaller trees
    n_estimators=5,  # Fewer bags for speed
    random_state=42,
    n_jobs=-1
)

start_time = time.time()
bagging_lgb.fit(X_train.values, y_train)
bagging_lgb_time = time.time() - start_time

y_pred_bagging_lgb = bagging_lgb.predict_proba(X_test.values)[:, 1]
bagging_lgb_auc = roc_auc_score(y_test, y_pred_bagging_lgb)

ensemble_models['BaggingLGB'] = {
    'model': bagging_lgb,
    'auc': bagging_lgb_auc,
    'train_time': bagging_lgb_time,
    'data_type': 'unscaled'
}

print(f"   Bagging LGB AUC: {bagging_lgb_auc:.3f} (Time: {bagging_lgb_time:.1f}s)")


In [ ]:
# 5. Weighted Ensemble (Custom - Very Fast)
print("\n5️⃣ Training Weighted Ensemble...")

# Fast individual models for weighting
individual_models = {
    'lgb': LGBMClassifier(random_state=42, verbosity=-1, n_estimators=100),
    'xgb': XGBClassifier(random_state=42, eval_metric='logloss', n_estimators=100),
    'sgd': SGDClassifier(random_state=42, loss='log_loss', max_iter=1000),
    'ada': AdaBoostClassifier(random_state=42, n_estimators=100)
}

model_weights = {}
individual_predictions = {}

start_time = time.time()
for name, model in individual_models.items():
    # Quick CV score for weighting (3-fold for speed)
    cv_scores = cross_val_score(model, X_train.values, y_train, cv=3, scoring='roc_auc')
    model_weights[name] = cv_scores.mean()

    # Train and predict
    model.fit(X_train.values, y_train)
    if hasattr(model, 'predict_proba'):
        individual_predictions[name] = model.predict_proba(X_test.values)[:, 1]
    elif hasattr(model, 'decision_function'):
        individual_predictions[name] = model.decision_function(X_test.values)

weighted_time = time.time() - start_time

# Normalize weights
total_weight = sum(model_weights.values())
model_weights = {k: v/total_weight for k, v in model_weights.items()}

# Weighted average prediction
y_pred_weighted = sum(model_weights[name] * pred
                    for name, pred in individual_predictions.items())

weighted_auc = roc_auc_score(y_test, y_pred_weighted)

ensemble_models['WeightedEnsemble'] = {
    'model': None,  # Custom ensemble
    'auc': weighted_auc,
    'train_time': weighted_time,
    'weights': model_weights,
    'data_type': 'unscaled'
}

print(f"   Weighted Ensemble AUC: {weighted_auc:.3f} (Time: {weighted_time:.1f}s)")
print(f"   Model weights: {model_weights}")

# 6. Bonus: Super Fast Ensemble (SGD + LightGBM only)
print("\n6️⃣ Training Super Fast Ensemble (SGD + LGB)...")

super_fast_models = [
    ('sgd', SGDClassifier(random_state=42, loss='log_loss', max_iter=500)),
    ('lgb', LGBMClassifier(random_state=42, verbosity=-1, n_estimators=50))
]

super_fast_voting = VotingClassifier(
    estimators=super_fast_models,
    voting='soft',
    n_jobs=-1
)

start_time = time.time()
super_fast_voting.fit(X_train.values, y_train)
super_fast_time = time.time() - start_time

y_pred_super_fast = super_fast_voting.predict_proba(X_test.values)[:, 1]
super_fast_auc = roc_auc_score(y_test, y_pred_super_fast)

ensemble_models['SuperFast'] = {
    'model': super_fast_voting,
    'auc': super_fast_auc,
    'train_time': super_fast_time,
    'data_type': 'unscaled'
}

print(f"   Super Fast AUC: {super_fast_auc:.3f} (Time: {super_fast_time:.1f}s)")

# Results Summary
print(f"\n📊 Fast Ensemble Methods Results:")
print("="*70)
print(f"{'Model':<20} {'AUC':<8} {'Train Time':<12} {'vs Best Individual':<15} {'Speed Grade'}")
print("-"*70)

best_individual_auc = 0.618  # From your results
for name, results in sorted(ensemble_models.items(), key=lambda x: x[1]['auc'], reverse=True):
    improvement = results['auc'] - best_individual_auc
    train_time_str = f"{results['train_time']:.1f}s" if results['train_time'] > 0 else "Instant"

    # Speed grading
    if results['train_time'] < 5:
        speed_grade = "🚀 Excellent"
    elif results['train_time'] < 15:
        speed_grade = "⚡ Good"
    else:
        speed_grade = "🐌 Slow"

    print(f"{name:<20} {results['auc']:<8.3f} {train_time_str:<12} {improvement:+.3f} {'':10} {speed_grade}")

# Best ensemble
best_ensemble = max(ensemble_models.items(), key=lambda x: x[1]['auc'])
print(f"\n🏆 Best Fast Ensemble: {best_ensemble[0]} (AUC: {best_ensemble[1]['auc']:.3f})")

# Business metrics for best ensemble
if best_ensemble[1]['model'] is not None:
    print(f"\n🎯 Business Impact - Best Fast Ensemble ({best_ensemble[0]}):")
    ensemble_business_metrics = evaluate_business_metrics(
        best_ensemble[1]['model'],
        X_test.values,
        y_test,
        f"{best_ensemble[0]} Ensemble"
    )

print(f"\n⚡ Speed Summary:")
print(f"   Fastest: SuperFast (~{ensemble_models['SuperFast']['train_time']:.1f}s)")
print(f"   Best Performance: {best_ensemble[0]} ({best_ensemble[1]['auc']:.3f} AUC)")

Even after trying these ensemble methods, Ridge still comes out on top. This is our prod model!

## Inference Latency Benchmarking

Understanding latency performance for production deployment

In [ ]:
# Get our final Ridge model
ridge_model = tuned_models['Ridge']['model']  # Or use your best Ridge model
scaler = preprocessor  # The ColumnTransformer we used

print(f"🎯 Model: Ridge Classifier (Production Candidate)")
print(f"   Parameters: {ridge_model.get_params()}")
print(f"   Model size: {sys.getsizeof(ridge_model)} bytes")

# 1. Single Prediction Latency (Most Important for BNPL)
print(f"\n1️⃣ Single Prediction Latency Test:")
print("-" * 40)

# Prepare single sample for prediction
single_sample = X_test.iloc[[0]]  # First test sample
single_sample_scaled = scaler.transform(single_sample)

# Warm up (JIT compilation, cache loading)
for _ in range(10):
    _ = ridge_model.decision_function(single_sample_scaled)

# Time single predictions
single_times = []
n_single_tests = 1000

start_time = time.time()
for i in range(n_single_tests):
    pred_start = time.perf_counter()
    prediction = ridge_model.decision_function(single_sample_scaled)
    pred_end = time.perf_counter()
    single_times.append((pred_end - pred_start) * 1000)  # Convert to ms

total_time = time.time() - start_time

# Statistics
single_mean = np.mean(single_times)
single_median = np.median(single_times)
single_p95 = np.percentile(single_times, 95)
single_p99 = np.percentile(single_times, 99)
single_max = np.max(single_times)

print(f"Single prediction latency (1000 tests):")
print(f"   Mean: {single_mean:.3f} ms")
print(f"   Median: {single_median:.3f} ms")
print(f"   95th percentile: {single_p95:.3f} ms")
print(f"   99th percentile: {single_p99:.3f} ms")
print(f"   Max: {single_max:.3f} ms")
print(f"   Target: <100 ms → {'✅ PASS' if single_p99 < 100 else '❌ FAIL'}")

In [ ]:
# 2. Batch Prediction Latency (For high volume scenarios)
print(f"\n2️⃣ Batch Prediction Latency Test:")
print("-" * 40)

batch_sizes = [1, 10, 100, 1000, 5000]
batch_results = {}

for batch_size in batch_sizes:
    if batch_size <= len(X_test):
        batch_sample = X_test.iloc[:batch_size]
        batch_sample_scaled = scaler.transform(batch_sample)

        # Warm up
        for _ in range(5):
            _ = ridge_model.decision_function(batch_sample_scaled)

        # Time batch predictions
        batch_times = []
        n_batch_tests = max(10, 100 // batch_size)  # Fewer tests for larger batches

        for _ in range(n_batch_tests):
            start = time.perf_counter()
            predictions = ridge_model.decision_function(batch_sample_scaled)
            end = time.perf_counter()
            batch_times.append(end - start)

        avg_batch_time = np.mean(batch_times)
        per_sample_time = (avg_batch_time / batch_size) * 1000  # ms per sample

        batch_results[batch_size] = {
            'total_time_ms': avg_batch_time * 1000,
            'per_sample_ms': per_sample_time,
            'throughput_per_sec': batch_size / avg_batch_time
        }

        print(f"Batch size {batch_size:4d}: {per_sample_time:.3f} ms/sample, "
            f"{batch_results[batch_size]['throughput_per_sec']:8.0f} predictions/sec")

In [ ]:
# 3. Preprocessing Overhead Analysis
print(f"\n3️⃣ Preprocessing Overhead Analysis:")
print("-" * 40)

# Time just preprocessing
preprocess_times = []
for _ in range(100):
    start = time.perf_counter()
    scaled_sample = scaler.transform(single_sample)
    end = time.perf_counter()
    preprocess_times.append((end - start) * 1000)

preprocess_mean = np.mean(preprocess_times)
preprocess_p95 = np.percentile(preprocess_times, 95)

# Time just model prediction (on pre-scaled data)
model_times = []
for _ in range(100):
    start = time.perf_counter()
    prediction = ridge_model.decision_function(single_sample_scaled)
    end = time.perf_counter()
    model_times.append((end - start) * 1000)

model_mean = np.mean(model_times)
model_p95 = np.percentile(model_times, 95)

print(f"Preprocessing time: {preprocess_mean:.3f} ms (95th: {preprocess_p95:.3f} ms)")
print(f"Model inference time: {model_mean:.3f} ms (95th: {model_p95:.3f} ms)")
print(f"Total pipeline time: {preprocess_mean + model_mean:.3f} ms")

In [ ]:
# 4. Memory Usage Analysis
print(f"\n4️⃣ Memory Usage Analysis:")
print("-" * 40)

import sys
import pickle

# Model size
model_size = sys.getsizeof(pickle.dumps(ridge_model))
scaler_size = sys.getsizeof(pickle.dumps(scaler))
total_size = model_size + scaler_size

print(f"Ridge model size: {model_size:,} bytes ({model_size/1024:.1f} KB)")
print(f"Scaler size: {scaler_size:,} bytes ({scaler_size/1024:.1f} KB)")
print(f"Total pipeline size: {total_size:,} bytes ({total_size/1024:.1f} KB)")
print(f"Memory efficiency: {'✅ Excellent' if total_size < 1024*1024 else '⚠️ Large'}")

In [ ]:
# 5. Production Readiness Assessment
print(f"\n5️⃣ Production Readiness Assessment:")
print("="*50)

# Requirements check
requirements = {
    'Inference Latency': {
        'requirement': '<100ms',
        'actual': f'{single_p99:.1f}ms (99th percentile)',
        'status': '✅ PASS' if single_p99 < 100 else '❌ FAIL'
    },
    'Model Size': {
        'requirement': '<50MB',
        'actual': f'{total_size/1024/1024:.1f}MB',
        'status': '✅ PASS' if total_size < 50*1024*1024 else '❌ FAIL'
    },
    'Throughput': {
        'requirement': '100K+ predictions/min',
        'actual': f'{batch_results[1000]["throughput_per_sec"]*60:,.0f} predictions/min',
        'status': '✅ PASS' if batch_results[1000]["throughput_per_sec"]*60 > 100000 else '❌ FAIL'
    }
}

for metric, details in requirements.items():
    print(f"{metric:15} | {details['requirement']:12} | {details['actual']:20} | {details['status']}")

In [ ]:
# 6. Latency Distribution Visualization
print(f"\n6️⃣ Latency Distribution Visualization:")
print("-" * 40)

plt.figure(figsize=(12, 8))

# Plot 1: Single prediction latency histogram
plt.subplot(2, 2, 1)
plt.hist(single_times, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(single_mean, color='red', linestyle='--', label=f'Mean: {single_mean:.2f}ms')
plt.axvline(single_p95, color='orange', linestyle='--', label=f'95th: {single_p95:.2f}ms')
plt.axvline(100, color='green', linestyle='-', linewidth=2, label='100ms Requirement')
plt.xlabel('Latency (ms)')
plt.ylabel('Frequency')
plt.title('Single Prediction Latency Distribution')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Batch size vs latency per sample
plt.subplot(2, 2, 2)
batch_sizes_plot = list(batch_results.keys())
per_sample_times = [batch_results[bs]['per_sample_ms'] for bs in batch_sizes_plot]
plt.plot(batch_sizes_plot, per_sample_times, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Batch Size')
plt.ylabel('Latency per Sample (ms)')
plt.title('Batch Size vs Per-Sample Latency')
plt.xscale('log')
plt.grid(True, alpha=0.3)

# Plot 3: Throughput vs batch size
plt.subplot(2, 2, 3)
throughputs = [batch_results[bs]['throughput_per_sec'] for bs in batch_sizes_plot]
plt.plot(batch_sizes_plot, throughputs, 'go-', linewidth=2, markersize=8)
plt.xlabel('Batch Size')
plt.ylabel('Throughput (predictions/sec)')
plt.title('Throughput vs Batch Size')
plt.xscale('log')
plt.yscale('log')
plt.grid(True, alpha=0.3)

# Plot 4: Component breakdown
plt.subplot(2, 2, 4)
components = ['Preprocessing', 'Model Inference']
times = [preprocess_mean, model_mean]
colors = ['lightblue', 'lightcoral']
plt.bar(components, times, color=colors, edgecolor='black')
plt.ylabel('Time (ms)')
plt.title('Pipeline Component Breakdown')
plt.grid(True, alpha=0.3, axis='y')

# Add values on bars
for i, (comp, time_val) in enumerate(zip(components, times)):
    plt.text(i, time_val + 0.001, f'{time_val:.3f}ms', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# 7. Production Deployment Summary
print(f"\n7️⃣ Production Deployment Summary:")
print("="*50)

if single_p99 < 100 and total_size < 50*1024*1024:
    print("🎉 Ridge Model APPROVED for Production Deployment!")
    print(f"   ✅ Meets all latency requirements ({single_p99:.1f}ms < 100ms)")
    print(f"   ✅ Lightweight model ({total_size/1024:.1f}KB << 50MB)")
    print(f"   ✅ High throughput capability ({batch_results[1000]['throughput_per_sec']:,.0f} pred/sec)")
    print(f"   ✅ Minimal infrastructure requirements")
else:
    print("⚠️ Ridge Model needs optimization for production:")
    if single_p99 >= 100:
        print(f"   ❌ Latency too high: {single_p99:.1f}ms")
    if total_size >= 50*1024*1024:
        print(f"   ❌ Model too large: {total_size/1024/1024:.1f}MB")

# Save benchmarking results
benchmark_results = {
    'model_type': 'RidgeClassifier',
    'single_prediction_latency_ms': {
        'mean': single_mean,
        'p95': single_p95,
        'p99': single_p99,
        'max': single_max
    },
    'batch_performance': batch_results,
    'preprocessing_overhead_ms': preprocess_mean,
    'model_inference_ms': model_mean,
    'model_size_bytes': total_size,
    'production_ready': single_p99 < 100 and total_size < 50*1024*1024
}

print(f"\n💾 Benchmark results saved for production planning")

## Feature Importance Review

In [ ]:
# Feature Importance Analysis for Ridge Model

# Get our final Ridge model and feature names
ridge_model = tuned_models['Ridge']['model']
feature_names = X_train.columns.tolist()

print(f"🎯 Model: Ridge Classifier")
print(f"   Features analyzed: {len(feature_names)}")
print(f"   Model parameters: {ridge_model.get_params()}")

# 1. Linear Coefficients Analysis (Ridge-specific)
print(f"\n1️⃣ Linear Coefficients Analysis:")
print("-" * 40)

# Get coefficients
coefficients = ridge_model.coef_[0]  # Ridge is binary classifier

# Create feature importance dataframe
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients),
    'importance_rank': range(1, len(feature_names) + 1)
}).sort_values('abs_coefficient', ascending=False).reset_index(drop=True)

feature_importance_df['importance_rank'] = range(1, len(feature_importance_df) + 1)

print(f"Top 10 Most Important Features (by absolute coefficient):")
print(feature_importance_df.head(10)[['feature', 'coefficient', 'abs_coefficient']].to_string(index=False))

print(f"\nCoefficient Statistics:")
print(f"   Range: [{coefficients.min():.4f}, {coefficients.max():.4f}]")
print(f"   Mean absolute: {np.abs(coefficients).mean():.4f}")
print(f"   Std: {coefficients.std():.4f}")

In [ ]:
# 2. Permutation Importance (Model-agnostic)
print(f"\n2️⃣ Permutation Importance Analysis:")
print("-" * 40)

# Calculate permutation importance on test set
print("Calculating permutation importance (this may take a moment)...")
perm_importance = permutation_importance(
    ridge_model, X_test_processed, y_test,
    n_repeats=10, random_state=42,
    scoring='roc_auc', n_jobs=-1
)

# Create permutation importance dataframe
perm_df = pd.DataFrame({
    'feature': feature_names,
    'perm_importance': perm_importance.importances_mean,
    'perm_std': perm_importance.importances_std
}).sort_values('perm_importance', ascending=False).reset_index(drop=True)

print(f"Top 10 Features by Permutation Importance:")
print(perm_df.head(10)[['feature', 'perm_importance', 'perm_std']].to_string(index=False, float_format='%.4f'))

In [ ]:
# 3. Feature Categories Analysis
print(f"\n3️⃣ Feature Categories Analysis:")
print("-" * 40)

# Categorize features
def categorize_feature(feature_name):
    if any(x in feature_name for x in ['customer_credit_score', 'customer_age', 'customer_income', 'customer_verification']):
        return 'Customer Demographics'
    elif any(x in feature_name for x in ['device_type', 'device_is_trusted']):
        return 'Device/Technology'
    elif any(x in feature_name for x in ['payment_provider', 'payment_credit_limit']):
        return 'Payment Context'
    elif any(x in feature_name for x in ['product_category', 'product_risk']):
        return 'Product Features'
    elif any(x in feature_name for x in ['purchase_context', 'price_comparison']):
        return 'Purchase Behavior'
    elif any(x in feature_name for x in ['risk_score', 'risk_scenario', 'risk_level']):
        return 'Risk Assessment'
    elif any(x in feature_name for x in ['is_weekend', 'is_month_end', 'is_holiday', 'is_business', 'is_late', 'time_of_day', 'week_of_month']):
        return 'Temporal Features'
    elif feature_name in ['amount', 'installment_count', 'customer_tenure_days']:
        return 'Transaction Details'
    else:
        return 'Other'

# Add categories to importance dataframes
feature_importance_df['category'] = feature_importance_df['feature'].apply(categorize_feature)
perm_df['category'] = perm_df['feature'].apply(categorize_feature)

# Category-wise importance
category_importance = feature_importance_df.groupby('category').agg({
    'abs_coefficient': ['mean', 'sum', 'count'],
    'coefficient': ['mean']
}).round(4)

category_importance.columns = ['avg_abs_coef', 'total_abs_coef', 'feature_count', 'avg_coef']
category_importance = category_importance.sort_values('total_abs_coef', ascending=False)

print("Feature Category Importance Summary:")
print(category_importance.to_string())

In [ ]:
# 4. Visualization Dashboard
print(f"\n4️⃣ Feature Importance Visualizations:")
print("-" * 40)

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Ridge Model Feature Importance Analysis Dashboard', fontsize=16, fontweight='bold')

# Plot 1: Top 15 coefficients
ax1 = axes[0, 0]
top_15 = feature_importance_df.head(15)
colors = ['red' if x < 0 else 'blue' for x in top_15['coefficient']]
bars = ax1.barh(range(len(top_15)), top_15['coefficient'], color=colors, alpha=0.7)
ax1.set_yticks(range(len(top_15)))
ax1.set_yticklabels(top_15['feature'], fontsize=8)
ax1.set_xlabel('Coefficient Value')
ax1.set_title('Top 15 Features by Coefficient')
ax1.axvline(x=0, color='black', linestyle='-', alpha=0.3)
ax1.grid(True, alpha=0.3)

# Plot 2: Permutation importance
ax2 = axes[0, 1]
top_15_perm = perm_df.head(15)
bars2 = ax2.barh(range(len(top_15_perm)), top_15_perm['perm_importance'],
                xerr=top_15_perm['perm_std'], alpha=0.7, color='green')
ax2.set_yticks(range(len(top_15_perm)))
ax2.set_yticklabels(top_15_perm['feature'], fontsize=8)
ax2.set_xlabel('Permutation Importance')
ax2.set_title('Top 15 Features by Permutation Importance')
ax2.grid(True, alpha=0.3)

# Plot 3: Category-wise importance
ax3 = axes[0, 2]
category_plot = category_importance.sort_values('total_abs_coef', ascending=True)
ax3.barh(range(len(category_plot)), category_plot['total_abs_coef'], alpha=0.7, color='orange')
ax3.set_yticks(range(len(category_plot)))
ax3.set_yticklabels(category_plot.index, fontsize=10)
ax3.set_xlabel('Total Absolute Coefficient')
ax3.set_title('Feature Category Importance')
ax3.grid(True, alpha=0.3)

# Plot 4: Coefficient distribution
ax4 = axes[1, 0]
ax4.hist(coefficients, bins=30, alpha=0.7, edgecolor='black', color='purple')
ax4.axvline(coefficients.mean(), color='red', linestyle='--', label=f'Mean: {coefficients.mean():.3f}')
ax4.axvline(0, color='black', linestyle='-', alpha=0.5)
ax4.set_xlabel('Coefficient Value')
ax4.set_ylabel('Frequency')
ax4.set_title('Coefficient Distribution')
ax4.legend()
ax4.grid(True, alpha=0.3)

# Plot 5: Coefficient vs Permutation Importance
ax5 = axes[1, 1]
comparison_df = pd.merge(feature_importance_df[['feature', 'abs_coefficient']],
                        perm_df[['feature', 'perm_importance']], on='feature')
ax5.scatter(comparison_df['abs_coefficient'], comparison_df['perm_importance'], alpha=0.6, s=50)

# Add correlation line
from scipy.stats import pearsonr
corr, p_value = pearsonr(comparison_df['abs_coefficient'], comparison_df['perm_importance'])
ax5.set_xlabel('Absolute Coefficient')
ax5.set_ylabel('Permutation Importance')
ax5.set_title(f'Coefficient vs Permutation Importance\n(Correlation: {corr:.3f})')
ax5.grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(comparison_df['abs_coefficient'], comparison_df['perm_importance'], 1)
p = np.poly1d(z)
ax5.plot(comparison_df['abs_coefficient'], p(comparison_df['abs_coefficient']), "r--", alpha=0.8)

# Plot 6: Feature importance by category (box plot)
ax6 = axes[1, 2]
category_data = []
category_labels = []
for cat in feature_importance_df['category'].unique():
    cat_data = feature_importance_df[feature_importance_df['category'] == cat]['abs_coefficient']
    category_data.append(cat_data)
    category_labels.append(cat)

ax6.boxplot(category_data, labels=category_labels)
ax6.set_xticklabels(category_labels, rotation=45, ha='right', fontsize=8)
ax6.set_ylabel('Absolute Coefficient')
ax6.set_title('Feature Importance Distribution by Category')
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 5. Business Insights
print(f"\n5️⃣ Business Insights from Feature Analysis:")
print("="*50)

# Most predictive features
top_business_features = feature_importance_df.head(10)
print("🔍 Top Predictive Features for BNPL Default Risk:")
for i, row in top_business_features.iterrows():
    direction = "increases" if row['coefficient'] > 0 else "decreases"
    print(f"   {i+1}. {row['feature']}: {direction} default risk (coef: {row['coefficient']:.4f})")

# Category insights
print(f"\n📊 Category-Level Insights:")
for category, data in category_importance.iterrows():
    avg_impact = "High" if data['avg_abs_coef'] > coefficients.std() else "Medium" if data['avg_abs_coef'] > coefficients.std()/2 else "Low"
    print(f"   {category}: {avg_impact} impact ({data['feature_count']} features, avg coef: {data['avg_abs_coef']:.4f})")

# Model interpretability summary
print(f"\n🎯 Model Interpretability Summary:")
print(f"   Total features: {len(feature_names)}")
print(f"   Features with |coef| > {coefficients.std():.4f}: {sum(feature_importance_df['abs_coefficient'] > coefficients.std())}")
print(f"   Most important category: {category_importance.index[0]}")
print(f"   Feature correlation with perm. importance: {corr:.3f} (p={p_value:.3f})")

# Save feature importance results
feature_analysis_results = {
    'model_type': 'RidgeClassifier',
    'top_features': top_business_features[['feature', 'coefficient', 'abs_coefficient']].to_dict('records'),
    'category_importance': category_importance.to_dict('index'),
    'coefficient_stats': {
        'mean': float(coefficients.mean()),
        'std': float(coefficients.std()),
        'min': float(coefficients.min()),
        'max': float(coefficients.max())
    },
    'correlation_coef_vs_perm': float(corr)
}

print(f"\n💾 Feature analysis results saved for production documentation")

## Further Model Eval

Before exporting model artifacts for production, it became apparent that we were leaving some business usefulness on the table

In [ ]:
# Business-Focused Model Evaluation for BNPL Default Risk
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

# Business cost parameters
AVG_TRANSACTION_AMOUNT = 144  
AVG_DOWNPAYMENT_RATE = 0.25   # 25% downpayment typical for BNPL
CHURN_PROBABILITY = 0.15      # 20% customers abandon purchase if BNPL rejected. Potentially lower because of the AOV, which is >2X Amazon's, pointing to less price sensitivity.

# Cost calculations
def calculate_business_costs(amount=AVG_TRANSACTION_AMOUNT):
    missed_default_cost = amount * (1 - AVG_DOWNPAYMENT_RATE)  # Loss if we lend to defaulter
    rejected_good_customer_cost = amount * CHURN_PROBABILITY    # Revenue loss if we reject good customer
    return missed_default_cost, rejected_good_customer_cost

missed_default_cost, rejected_good_cost = calculate_business_costs()
print(f"📊 Business Cost Structure:")
print(f"   • Cost of missed default: ${missed_default_cost:.2f}")
print(f"   • Cost of rejected good customer: ${rejected_good_cost:.2f}")
print(f"   • Cost ratio (FN:FP): {missed_default_cost/rejected_good_cost:.2f}:1")

In [ ]:
def evaluate_business_metrics_comprehensive(model, X_test, y_test, model_name):
    """
    Comprehensive business evaluation for BNPL models
    Handles both probability-based and score-based models
    """
    # Get predictions - handle RidgeClassifier vs LogisticRegression/ElasticNet
    if hasattr(model, 'predict_proba'):
        # LogisticRegression and ElasticNet have predict_proba
        y_pred_proba = model.predict_proba(X_test)[:, 1]
    elif hasattr(model, 'decision_function'):
        # RidgeClassifier has decision_function - convert to probabilities
        decision_scores = model.decision_function(X_test)
        # Convert decision scores to probabilities using sigmoid
        from scipy.special import expit
        y_pred_proba = expit(decision_scores)
    else:
        raise ValueError(f"Model {model_name} doesn't support probability predictions")

    # Calculate precision-recall curve
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)

    # Key metrics
    auc_pr = average_precision_score(y_test, y_pred_proba)

    # Business-critical metrics at different precision levels
    results = {
        'model_name': model_name,
        'auc_pr': auc_pr,
    }

    # Find recall at specific precision thresholds
    # Note: thresholds array is one element shorter than precision/recall
    for target_precision in [0.90, 0.95, 0.98]:
        valid_indices = precision[:-1] >= target_precision  # Exclude last element to match thresholds
        if valid_indices.any():
            valid_recall = recall[:-1][valid_indices]  # Exclude last element
            max_recall = valid_recall.max()
            optimal_threshold_idx = valid_recall.argmax()
            optimal_threshold = thresholds[valid_indices][optimal_threshold_idx]
        else:
            max_recall = 0.0
            optimal_threshold = 1.0

        results[f'recall_at_{int(target_precision*100)}_precision'] = max_recall
        results[f'threshold_for_{int(target_precision*100)}_precision'] = optimal_threshold

    # Find precision at specific recall thresholds
    for target_recall in [0.70, 0.80, 0.90]:
        valid_indices = recall[:-1] >= target_recall  # Exclude last element to match thresholds
        if valid_indices.any():
            valid_precision = precision[:-1][valid_indices]  # Exclude last element
            max_precision = valid_precision.max()
            optimal_threshold_idx = valid_precision.argmax()
            optimal_threshold = thresholds[valid_indices][optimal_threshold_idx]
        else:
            max_precision = 0.0
            optimal_threshold = 0.0

        results[f'precision_at_{int(target_recall*100)}_recall'] = max_precision
        results[f'threshold_for_{int(target_recall*100)}_recall'] = optimal_threshold

    # Expected business value calculation
    # Find threshold that maximizes business value
    best_value = -float('inf')
    best_threshold = 0.5
    best_precision = 0
    best_recall = 0
    best_f1 = 0

    for threshold in np.linspace(0.1, 0.9, 50):
        y_pred_binary = (y_pred_proba >= threshold).astype(int)
        cm = confusion_matrix(y_test, y_pred_binary)

        if cm.shape == (2, 2):
            tn, fp, fn, tp = cm.ravel()

            # Business value calculation
            # Revenue saved by catching defaults - revenue lost by rejecting good customers
            value_saved = tp * missed_default_cost  # True positives: defaults caught
            value_lost = fp * rejected_good_cost    # False positives: good customers rejected
            net_business_value = value_saved - value_lost

            if net_business_value > best_value:
                best_value = net_business_value
                best_threshold = threshold
                best_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
                best_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
                best_f1 = 2 * (best_precision * best_recall) / (best_precision + best_recall) if (best_precision + best_recall) > 0 else 0

    results.update({
        'optimal_business_threshold': best_threshold,
        'max_business_value': best_value,
        'precision_at_optimal_threshold': best_precision,
        'recall_at_optimal_threshold': best_recall,
        'f1_at_optimal_threshold': best_f1
    })

    return results

print("✅ Business evaluation function defined (fixed array dimension issue)")

In [ ]:
# Evaluate all models with business metrics
models_to_evaluate = {
    'Ridge': tuned_models['Ridge']['model'],
    'LogisticRegression': tuned_models['LogisticRegression']['model'],
    'ElasticNet': tuned_models['ElasticNet']['model']
}

business_results = {}

print("🔄 Evaluating models with business metrics...")
print()

for model_name, model in models_to_evaluate.items():
    print(f"Evaluating {model_name}...")

    # Use scaled data for linear models
    X_eval = X_test_processed

    results = evaluate_business_metrics_comprehensive(model, X_eval, y_test, model_name)
    business_results[model_name] = results

    print(f"✅ {model_name} evaluation complete")

print(f"\n📊 Business evaluation complete for {len(models_to_evaluate)} models")

In [ ]:
# Create comprehensive comparison table
# Convert results to DataFrame
comparison_df = pd.DataFrame(business_results).T

# Key business metrics for ranking
key_metrics = [
    'auc_pr',
    'recall_at_95_precision',
    'recall_at_90_precision',
    'precision_at_80_recall',
    'max_business_value',
    'optimal_business_threshold',
    'f1_at_optimal_threshold'
]

print("🏆 BUSINESS METRICS COMPARISON")
print("=" * 60)

# Display comparison table
display_df = comparison_df[key_metrics].round(4)
print(display_df.to_string())

print(f"\n💰 BUSINESS VALUE ANALYSIS")
print("-" * 40)
for model_name in business_results.keys():
    value = business_results[model_name]['max_business_value']
    threshold = business_results[model_name]['optimal_business_threshold']
    precision = business_results[model_name]['precision_at_optimal_threshold']
    recall = business_results[model_name]['recall_at_optimal_threshold']

    print(f"{model_name}:")
    print(f"  • Max Business Value: ${value:.2f}")
    print(f"  • Optimal Threshold: {threshold:.3f}")
    print(f"  • Precision at Optimal: {precision:.3f}")
    print(f"  • Recall at Optimal: {recall:.3f}")
    print()

In [ ]:
# DIAGNOSTIC CELL - Add this to investigate the issue
print("🔍 DIAGNOSTIC ANALYSIS")
print("=" * 50)

# Check basic model performance
for model_name, model in models_to_evaluate.items():
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test_processed)[:, 1]
    elif hasattr(model, 'decision_function'):
        decision_scores = model.decision_function(X_test_processed)
        from scipy.special import expit
        y_pred_proba = expit(decision_scores)

    y_pred_binary = model.predict(X_test_processed)

    # Basic metrics
    from sklearn.metrics import accuracy_score, roc_auc_score
    accuracy = accuracy_score(y_test, y_pred_binary)
    auc_roc = roc_auc_score(y_test, y_pred_proba)

    print(f"\n{model_name}:")
    print(f"  • Accuracy: {accuracy:.3f}")
    print(f"  • AUC-ROC: {auc_roc:.3f}")
    print(f"  • Prediction range: {y_pred_proba.min():.3f} - {y_pred_proba.max():.3f}")
    print(f"  • Mean prediction: {y_pred_proba.mean():.3f}")
    print(f"  • Predictions > 0.5: {(y_pred_proba > 0.5).sum()}")

# Check test set composition
print(f"\n📊 Test Set Analysis:")
print(f"  • Total samples: {len(y_test)}")
print(f"  • Actual defaults: {y_test.sum()}")
print(f"  • Default rate: {y_test.mean():.3f}")

# Check if there's a class imbalance issue
print(f"\n⚠️  POTENTIAL ISSUES:")
if y_test.mean() < 0.02:
    print("  • Extremely low default rate - models may default to 'no default'")
if all([(y_pred_proba.max() < 0.5) for model_name, model in models_to_evaluate.items() for y_pred_proba in [model.predict_proba(X_test_processed)[:, 1] if hasattr(model, 'predict_proba') else
expit(model.decision_function(X_test_processed))]]):
    print("  • All models predict very low probabilities")

# Check feature scaling impact
print(f"\n🔧 Feature Analysis:")
print(f"  • X_test_processed shape: {X_test_processed.shape}")
print(f"  • X_test_processed range: {X_test_processed.min():.3f} - {X_test_processed.max():.3f}")

In [ ]:
# Test performance at appropriate thresholds for imbalanced data
print("🎯 IMBALANCED DATA APPROPRIATE THRESHOLDS")
print("=" * 50)

for model_name, model in models_to_evaluate.items():
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test_processed)[:, 1]
    elif hasattr(model, 'decision_function'):
        from scipy.special import expit
        y_pred_proba = expit(model.decision_function(X_test_processed))

    # Use percentile-based thresholds instead of absolute thresholds
    thresholds = [np.percentile(y_pred_proba, p) for p in [90, 95, 98, 99]]

    print(f"\n{model_name}:")
    for i, threshold in enumerate([90, 95, 98, 99]):
        thresh_val = thresholds[i]
        y_pred_binary = (y_pred_proba >= thresh_val).astype(int)

        if y_pred_binary.sum() > 0:  # Avoid division by zero
            precision = (y_pred_binary & y_test).sum() / y_pred_binary.sum()
            recall = (y_pred_binary & y_test).sum() / y_test.sum()
            print(f"  • Top {100-threshold}%: Precision={precision:.3f}, Recall={recall:.3f}, Threshold={thresh_val:.3f}")
        else:
            print(f"  • Top {100-threshold}%: No predictions above threshold")

In [ ]:
# Business-focused ranking with weighted criteria
print("🎯 BUSINESS-FOCUSED MODEL RANKING")
print("=" * 50)

# Define business priorities (weights sum to 1.0)
ranking_weights = {
    'recall_at_95_precision': 0.35,    # Primary: catch defaults conservatively  
    'auc_pr': 0.25,                    # Overall performance for imbalanced data
    'max_business_value': 0.25,        # Economic impact
    'recall_at_90_precision': 0.15,    # Secondary: less conservative recall
}

print("📋 Ranking Criteria:")
for metric, weight in ranking_weights.items():
    print(f"   • {metric}: {weight*100:.1f}%")

# Calculate weighted scores
model_scores = {}

for model_name in business_results.keys():
    score = 0
    for metric, weight in ranking_weights.items():
        metric_value = business_results[model_name][metric]
        # Normalize to 0-1 scale based on max value across models
        max_value = max([business_results[m][metric] for m in business_results.keys()])
        if max_value > 0:
            normalized_value = metric_value / max_value
        else:
            normalized_value = 0
        score += weight * normalized_value

    model_scores[model_name] = score

# Rank models
ranked_models = sorted(model_scores.items(), key=lambda x: x[1], reverse=True)

print(f"\n🏅 BUSINESS-FOCUSED RANKINGS:")
print("-" * 30)
for i, (model_name, score) in enumerate(ranked_models, 1):
    auc_pr = business_results[model_name]['auc_pr']
    recall_95 = business_results[model_name]['recall_at_95_precision']
    business_value = business_results[model_name]['max_business_value']

    status = "🥇 CHAMPION" if i == 1 else f"🥈 CHALLENGER #{i-1}"

    print(f"{i}. {model_name} - Score: {score:.3f} {status}")
    print(f"   • AUC-PR: {auc_pr:.3f}")
    print(f"   • Recall@95%Prec: {recall_95:.3f}")
    print(f"   • Business Value: ${business_value:.2f}")
    print()

# Update champion selection
business_champion = ranked_models[0][0]
print(f"🎯 BUSINESS CHAMPION: {business_champion}")
print(f"   (Previous AUC-ROC champion: Ridge)")

if business_champion != 'Ridge':
    print(f"⚠️  CHAMPION CHANGE DETECTED!")
    print(f"   Business metrics favor {business_champion} over Ridge")
else:
    print(f"✅ Ridge confirmed as champion by business metrics")

In [ ]:
# Create business metrics visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Precision-Recall Curves
ax1 = axes[0, 0]
for model_name, model in models_to_evaluate.items():
# Handle different model types
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test_processed)[:, 1]
    elif hasattr(model, 'decision_function'):
        decision_scores = model.decision_function(X_test_processed)
        from scipy.special import expit
        y_pred_proba = expit(decision_scores)
    
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    auc_pr = business_results[model_name]['auc_pr']
    ax1.plot(recall, precision, label=f'{model_name} (AUC-PR: {auc_pr:.3f})')

ax1.set_xlabel('Recall')
ax1.set_ylabel('Precision')
ax1.set_title('Precision-Recall Curves')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Business Value Comparison
ax2 = axes[0, 1]
models = list(business_results.keys())
values = [business_results[m]['max_business_value'] for m in models]
colors = ['gold' if m == business_champion else 'lightblue' for m in models]

bars = ax2.bar(models, values, color=colors, alpha=0.7)
ax2.set_ylabel('Max Business Value ($)')
ax2.set_title('Business Value by Model')
ax2.tick_params(axis='x', rotation=45)

# Annotate bars
for bar, value in zip(bars, values):
    ax2.annotate(f'${value:.0f}',
                xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom')

# Plot 3: Recall at Different Precision Thresholds
ax3 = axes[1, 0]
precision_levels = [90, 95, 98]
x = np.arange(len(precision_levels))
width = 0.25

for i, model_name in enumerate(models):
    recalls = [business_results[model_name][f'recall_at_{p}_precision'] for p in precision_levels]
    ax3.bar(x + i * width, recalls, width, label=model_name, alpha=0.7)

ax3.set_xlabel('Precision Threshold (%)')
ax3.set_ylabel('Recall')
ax3.set_title('Recall at High Precision Thresholds')
ax3.set_xticks(x + width)
ax3.set_xticklabels([f'{p}%' for p in precision_levels])
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Model Ranking Scores
ax4 = axes[1, 1]
models_ranked = [item[0] for item in ranked_models]
scores = [item[1] for item in ranked_models]
colors = ['gold', 'silver', 'chocolate'][:len(models_ranked)]

bars = ax4.bar(models_ranked, scores, color=colors, alpha=0.7)
ax4.set_ylabel('Composite Business Score')
ax4.set_title('Business-Focused Model Ranking')
ax4.tick_params(axis='x', rotation=45)

# Annotate champion
for bar, score in zip(bars, scores):
    ax4.annotate(f'{score:.3f}',
                xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom')

plt.tight_layout()
plt.suptitle('BNPL Model Business Performance Dashboard', fontsize=16, fontweight='bold', y=1.02)
plt.show()

print("✅ Business metrics visualization complete!")